# Setting Up

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
! pip install -q transformers[sentencepiece] fastai ohmeow-blurr nbdev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
! pip install -q onnxruntime onnx onnxruntime-gpu onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.9 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")


The repository for squad contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/squad.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Classifying Book Plot Summaries into Multiple Genres/ONNX Inference

/content/drive/MyDrive/Classifying Book Plot Summaries into Multiple Genres/ONNX Inference


# Data

In [ ]:
df = pd.read_csv("best_books_details.csv")
df.head()

,title,url,description,genres
0,The Invisible Life of Addie LaRue,https://www.goodreads.com/book/show/50623864-the-invisible-life-of-addie-larue,"France, 1714: in a moment of desperation, a young woman makes a Faustian bargain to live forever and is cursed to be forgotten by everyone she meets.Thus begins the extraordinary life of Addie LaRue, and a dazzling adventure that will play out across centuries and continents, across history and art, as a young woman learns how far she will go to leave her mark on the world.But everything changes when, after nearly 300 years, Addie stumbles across a young man in a hidden bookstore and he remembers her name.","['Fantasy', 'Fiction', 'Romance', 'Historical Fiction', 'Adult', 'Historical', 'Magical Realism', 'LGBT', 'Contemporary', 'Paranormal']"
1,Project Hail Mary,https://www.goodreads.com/book/show/54493401-project-hail-mary,"Ryland Grace is the sole survivor on a desperate, last-chance mission—and if he fails, humanity and the earth itself will perish.Except that right now, he doesn’t know that. He can’t even remember his own name, let alone the nature of his assignment or how to complete it.All he knows is that he’s been asleep for a very, very long time. And he’s just been awakened to find himself millions of miles from home, with nothing but two corpses for company.His crewmates dead, his memories fuzzily returning, Ryland realizes that an impossible task now confronts him. Hurtling through space on this ti...","['Science Fiction', 'Fiction', 'Audiobook', 'Fantasy', 'Space', 'Adult', 'Thriller', 'Science Fiction Fantasy', 'Adventure', 'Mystery']"
2,The Midnight Library,https://www.goodreads.com/book/show/52578297-the-midnight-library,"Between life and death there is a library.When Nora Seed finds herself in the Midnight Library, she has a chance to make things right. Up until now, her life has been full of misery and regret. She feels she has let everyone down, including herself. But things are about to change.The books in the Midnight Library enable Nora to live as if she had done things differently. With the help of an old friend, she can now undo every one of her regrets as she tries to work out her perfect life. But things aren't always what she imagined they'd be, and soon her choices place the library and herself ...","['Fiction', 'Fantasy', 'Contemporary', 'Audiobook', 'Magical Realism', 'Science Fiction', 'Adult', 'Mental Health', 'Book Club', 'Adult Fiction']"
3,I'm Glad My Mom Died,https://www.goodreads.com/book/show/59364173-i-m-glad-my-mom-died,"A heartbreaking and hilarious memoir by iCarly and Sam & Cat star Jennette McCurdy about her struggles as a former child actor—including eating disorders, addiction, and a complicated relationship with her overbearing mother—and how she retook control of her life.Jennette McCurdy was six years old when she had her first acting audition. Her mother’s dream was for her only daughter to become a star, and Jennette would do anything to make her mother happy. So she went along with what Mom called “calorie restriction,” eating little and weighing herself five times a day. She endured extensive ...","['Nonfiction', 'Memoir', 'Audiobook', 'Biography', 'Autobiography', 'Mental Health', 'Adult', 'Biography Memoir', 'Contemporary', 'Humor']"
4,Piranesi,https://www.goodreads.com/book/show/50202953-piranesi,"Piranesi’s house is no ordinary building: its rooms are infinite, its corridors endless, its walls are lined with thousands upon thousands of statues, each one different from all the others. Within the labyrinth of halls an ocean is imprisoned; waves thunder up staircases, rooms are flooded in an instant. But Piranesi is not afraid; he understands the tides as he understands the pattern of the labyrinth itself. He lives to explore the house.There is one other person in the house—a man called The Other, who visits Piranesi twice a week and asks for help with research into A Great and Secret...","['Fantasy', 'Fiction', 'Mystery', 'Magica

In [ ]:
df = df.dropna().reset_index(drop=True)
df.shape

(24754, 4)

In [ ]:
genres_list = df.genres.to_list()
genre_count = {}
for genres in genres_list:
  genre_list = eval(genres)
  for genre in genre_list:
    if genre in genre_count.keys():
      genre_count[genre] += 1
    else:
      genre_count[genre] = 1
print(f"Number of Genres: {len(genre_count)}")
print(genre_count)

threshold = int(len(df) * 0.01)
rare_genres = [key for key, value in genre_count.items() if value < threshold]
len(rare_genres)

Number of Genres: 910
{'Fantasy': 7270, 'Fiction': 17766, 'Romance': 5299, 'Historical Fiction': 5225, 'Adult': 5041, 'Historical': 4609, 'Magical Realism': 576, 'LGBT': 1085, 'Contemporary': 5009, 'Paranormal': 2038, 'Science Fiction': 3066, 'Audiobook': 7478, 'Space': 385, 'Thriller': 3036, 'Science Fiction Fantasy': 2262, 'Adventure': 3360, 'Mystery': 4923, 'Mental Health': 396, 'Book Club': 676, 'Adult Fiction': 2083, 'Nonfiction': 5057, 'Memoir': 1662, 'Biography': 2243, 'Autobiography': 793, 'Biography Memoir': 1295, 'Humor': 2451, 'Mythology': 502, 'Literary Fiction': 2827, 'Dragons': 321, 'Fantasy Romance': 213, 'New Adult': 454, 'Magic': 1996, 'Young Adult': 4720, 'Race': 447, 'Dystopia': 897, 'Teen': 955, 'Young Adult Fantasy': 678, 'Urban Fantasy': 1362, 'Angels': 113, 'Dark': 283, 'Abuse': 159, 'Feminism': 643, 'Contemporary Romance': 749, 'Chick Lit': 1135, 'Books About Books': 272, 'Love': 162, 'College': 137, 'Fae': 218, 'Family': 962, 'Coming Of Age': 623, 'Historical F

759

In [ ]:
genres_list = df.genres.to_list()
revised_genre_list = []
indices_to_drop = []

for idx, genres in enumerate(genres_list):
  genre_list = eval(genres)
  revised_genres = []

  for genre in genre_list:
    if genre not in rare_genres:
      revised_genres.append(genre)

  if len(revised_genres) == 0:
    indices_to_drop.append(idx)
  else:
    revised_genre_list.append(revised_genres)

df = df.drop(indices_to_drop).reset_index(drop=True)
df['revised_genres'] = revised_genre_list
df.shape

(24143, 5)

In [ ]:
revised_genres_list = df.revised_genres.to_list()
revised_genre_count = {}
for genres in revised_genres_list:
  genre_list = genres
  for genre in genre_list:
    if genre in revised_genre_count.keys():
      revised_genre_count[genre] += 1
    else:
      revised_genre_count[genre] = 1
print(f"Number of Genres: {len(revised_genre_count)}")

Number of Genres: 151


In [ ]:
encode_genre_types = { key: idx for idx, (key, value) in enumerate(revised_genre_count.items())}
with open("genre_types_encoded.json", "w") as fp:
  json.dump(encode_genre_types, fp)

# We need this because for multilabel classification all genres have possibility to be present in the predictions
categorical_genre_list = []
revised_genres_list = df.revised_genres.to_list()

for revised_genres in revised_genres_list:
  categorical_list = [0] * len(encode_genre_types)
  for genre in revised_genres:
    genre_type_index = encode_genre_types[genre]
    categorical_list[genre_type_index] = 1
  categorical_genre_list.append(categorical_list)

df['genre_cat_list'] = categorical_genre_list
df.shape

(24143, 6)

In [ ]:
labels = list(encode_genre_types.keys())
len(labels), labels[:5]

(151, ['Fantasy', 'Fiction', 'Romance', 'Historical Fiction', 'Adult'])

# Data Split

In [ ]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

(21729, 2414)

In [ ]:
valid_df = df.loc[valid_ids]
valid_df.head()

,title,url,description,genres,revised_genres,genre_cat_list
13595,Boltzmanns Atom: The Great Debate That Launched A Revolution In Physics,https://www.goodreads.com/book/show/309719.Boltzmanns_Atom,"A portrait of the Austrian physicist goes back to the turn of the century, when the existence of the atom was still largely theoretical and two men, Ludwig Boltzmann and Ernst Mach, competed for accolades amid a growing field of scientists.","['Science', 'Physics', 'Nonfiction', 'Biography', 'History']","[Science, Nonfiction, Biography, History]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
11532,Five Point Someone: What Not to Do at IIT,https://www.goodreads.com/book/show/105576.Five_Point_Someone,"Five Point Someone is a story about three friends in IIT who are unable to cope.The book starts with a disclaimer, “This is not a book to teach you how to get into IIT or even how to live in college. In fact, it describes how screwed up things can get if you don’t think straight.”Three hostelmates – Alok, Hari and Ryan get off to a bad start in IIT – they screw up the first class quiz. And while they try to make amends, things only get worse. It takes them a while to realize: If you try and screw with the IIT system, it comes back to double screw you. Before they know it, they are at the l...","['Fiction', 'Indian Literature', 'India', 'Romance', 'Contemporary', 'Novels', 'Young Adult', 'Humor', 'Drama', 'Comedy']","[Fiction, Romance, Contemporary, Novels, Young Adult, Humor, Drama, Comedy]","[0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...]"
5715,Stray,https://www.goodreads.com/book/show/10837174-stray,"On her last day of high school, Cassandra Devlin walked out of exams and into a forest. Surrounded by the wrong sort of trees, and animals never featured in any nature documentary, Cass is only sure of one alone, she will be lucky to survive. The sprawl of abandoned blockish buildings Cass discovers offers her only more puzzles. Where are the people? What is the intoxicating mist which drifts off the buildings in the moonlight? And why does she feel like she's being watched? Increasingly unnerved, Cass is overjoyed at the arrival of the formidable Setari. Whisked to a world as technologica...","['Science Fiction', 'Young Adult', 'Fantasy', 'Aliens', 'Fiction', 'Romance', 'Adventure', 'Paranormal', 'Science Fiction Fantasy', 'Young Adult Fantasy']","[Science Fiction, Young Adult, Fantasy, Aliens, Fiction, Romance, Adventure, Paranormal, Science Fiction Fantasy, Young Adult Fantasy]","[1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...]"
13377,Batman Unmasked: Analyzing a Cultural Icon,https://www.goodreads.com/book/show/506597.Batman_Unmasked,"Over the sixty years of his existence, Batman has encountered an impressive array of cultural icons and has gradually become one himself. This acclaimed book examines what Batman means and has meant to the various audiences, groups and communities who have tried to control and interpret him over the decades. Brooker reveals the struggles over Batman's meaning by shining a light on the cultural issues of the day that impacted on the development of the character. They include: patriotic propaganda of the Second World War; the accusation that Ba

# Fastai & Blurr Inference

In [ ]:
model_path = "models/book-classifier-stage-1.pkl"
learner_inf = load_learner(model_path)

In [ ]:
learner_inf.blurr_predict("random placeholder")

[{'labels': ['Fiction'],
  'scores': [0.8117122650146484],
  'class_indices': [0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'class_labels':

In [ ]:
learner_inf.blurr_predict("random placeholder")[0]['labels']

['Fiction']

## Evaluation

In [ ]:
from sklearn import metrics

def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df['genre_cat_list'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]


  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [ ]:
valid_df['description'] = valid_df['description'].fillna('')

preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
    desc = row['description']
    labels = learner_inf.blurr_predict(desc)[0]['labels']
    pred_genres = [0] * len(encode_genre_types)
    for label in labels:
        pred_genres[encode_genre_types[label]] = 1
    preds.append(pred_genres)

preds[0][:20]

  0%|          | 0/2414 [00:00<?, ?it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.7384650921060231
F1 Score (Macro) = 0.5210817797472947


##Convert to ONNX

In [ ]:
model_path = "models/book-classifier-stage-1.pkl"
learner_inf = load_learner(model_path)

In [ ]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    'models/book-classifier.onnx',
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_model_path = 'models/book-classifier.onnx'
quantized_onnx_model_path = 'models/book-classifier-quantized.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8,
)

# ONNX Inference

## Normal ONNX

In [ ]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession('models/book-classifier.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/2414 [00:00<?, ?it/s]

In [ ]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.7375086745315753
F1 Score (Macro) = 0.5211149556226734


## Quantized ONNX

In [ ]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession('models/book-classifier-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/2414 [00:00<?, ?it/s]

In [ ]:
metric_measures(valid_df, preds) #

F1 Score (Micro) = 0.7359319877392864
F1 Score (Macro) = 0.5189432743566849
